# MD004 ENTREGA6 ANÁLISIS DE COMPONENTES PRINCIPALES

**Descripción de la tarea:**

El fichero 20230119_DatosBoston incluye datos de precios del mercado de viviendas de Boston (median_value), así como datos adicionales sobre la ubicación de la vivienda y otros datos sociodemográficos. La descripción de las columnas del dataset es la que sigue:

- **crime_index**: Índice de criminalidad de la zona
- **urban_ratio**: Ratio de zona urbana residencial sobre el total
- **industry_ratio**: Ratio de industria de la zona
- **c_river_variable**: 1 si la zona colinda con un río y 0 si no
- **nox_concentration**: Concentración de óxido nitroso en la zona
- **rooms_per_dwelling**: Número de habitaciones medio por vivienda
- **dwelling_age**: “Edad” media de la vivienda
- **dwelling_dist**: Distancia media ponderada con los centros urbanos de la ciudad
- **highway_index**: Índice de accesibilidad sobre autopistas cercanas
- **tax_ratio**: Índice de tasas por cada 10000usd
- **teacher_ratio**: Ratio de alumno por profesor
- **racial_index**: Índice de proporción de población no caucásica
- **lower_status**: Porcentaje de población de “clase-baja”
- **median_value**: Valor medio de precio de la vivienda (En 1000USD)

**Se pide:**

1. **Visualización y familiarización del dataset (2p)**: Se valorará el uso de las técnicas empleadas, así como la explicación de las visualizaciones / estadísticos utilizados en clases anteriores.
2. **Análisis de correlación y de componentes principales (3p)**: Se valorará principalmente la interpretación de dichos análisis así como la “limpieza” de la presentación
3. **Análisis de ANOVA (2p)**: Se valorará principalmente la interpretación del análisis, así como la limpieza de los datos
4. **Desarrollo y justificación de las variables (1p)**: En base a las conclusiones sacadas en los puntos 1, 2, y 3 ¿Qué variables debemos utilizar en nuestro modelo de regresión lineal?
5. **Estudio de regresión lineal del median_value (2p)**: ¿Qué resultados obtenemos en el modelo?,  ¿Predice correctamente?, ¿Utilizarías este modelo en la vida real?  

<font color='red'> **Entrega: fichero Jupyter Notebook con MD004NombreApellidosAC06.ipynb y fichero .pdf con MD004NombreApellidosAC04.pdf** </font>

In [ ]:
install.packages("ggplot2")
install.packages("dplyr")
install.packages("tidyverse")
install.packages("corrplot")
install.packages(c("ggplot2", "dplyr"))
install.packages(c("GGally", "gridExtra"))
install.packages("reshape2")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(ggplot2)
library(GGally)
library(gridExtra)
library(dplyr)
library(corrplot)

In [ ]:
#install.packages("tidyverse")
install.packages('gridExtra')
#install.packages('nortest')
#install.packages ('GGally')
#install.packages("olsrr")
library(tidyverse)
library(dplyr)
library(MASS)
library(class)

library(ggplot2)
library(gridExtra)
library(GGally)

library(reshape2)



In [ ]:
data_boston <- read.csv("20230119_DatosBoston.csv", header=TRUE, sep=';', dec='.')

head(data_boston)
summary(data_boston)

## 1. Visualización y familiarización del dataset (2p):
Se valorará el uso de las técnicas empleadas, así como la explicación de las visualizaciones / estadísticos utilizados en clases anteriores.


In [ ]:
dim(data_boston)

In [ ]:
str(data_boston)

In [ ]:
sum(is.na(data_boston))# Núm de NA #comprobar si existen valores nulls

## Numeric variables

In [ ]:
data_boston_num = data_boston %>%
  select_if(is.numeric)

head(data_boston_num)

In [ ]:
summary(data_boston_num)

Calculation of descriptive statistics

In [ ]:

# Gráficos de histograma
graficos_histograma <- list()
variables_numericas <- c("crime_index", "urban_ratio", "industry_ratio", "c_river_variable",
                         "nox_concentration", "rooms_per_dwelling", "dwelling_age",
                         "dwelling_dist", "highway_index", "tax_ratio", "teacher_ratio",
                         "racial_index", "lower_status", "median_value")
var_data <- colnames(data_boston[, variables_numericas])

for (variable in var_data) {
  grafico <- ggplot(data_boston) +
    aes_string(x = variable) +
    geom_histogram(bins = 8, fill = '#e69138', color = 'white', na.rm = TRUE) +
    theme(text = element_text(size = 15), axis.text.x = element_text(size = 14)) +
    labs(title = paste('Hist', variable))

  graficos_histograma[[length(graficos_histograma) + 1]] <- grafico
}

options(
  repr.plot.width = 15,
  repr.plot.height = 10
)

do.call('grid.arrange', c(graficos_histograma, ncol = 3))


In [ ]:
box_plots <- list()

var_data <- colnames(data_boston[, variables_numericas])

for (variable in var_data) {
  box_plot <- ggplot(data_boston, aes(x = '', y = !!sym(variable))) +
    stat_boxplot(geom = 'errorbar', color = '#3c0023', na.rm = TRUE) +
    geom_boxplot(fill = '#e69138', color = '#3c0023', na.rm = TRUE) +
    coord_flip() +
    labs(title = paste('Boxplot', variable), y = variable) +
    theme(panel.background = element_rect(fill = 'lightgray'),
          panel.grid.major = element_line(color = 'white'),
          panel.grid.minor = element_line(color = 'white')) +
    theme(text = element_text(size = 12))
  box_plots[[variable]] <- box_plot
}

options(
  repr.plot.width = 15,
  repr.plot.height = 10
)
grid.arrange(grobs = box_plots, ncol = 3, widths = rep(2, 3))

In [ ]:
options(repr.plot.width = 15,repr.plot.height = 8)
qq_plots <- list()
var_data <- colnames(data_boston[, variables_numericas])

for (i in seq_along(var_data)) {
  variable <- var_data[i]
  qq_plot <- ggplot(data_boston[!is.na(data_boston[[variable]]), ],aes(sample = !!sym(variable))) +
    stat_qq() +
    stat_qq_line(color ='#e69138', lwd = 1) +
    labs(title = paste('qqplot', variable),
         x = 'Theoretical Quantiles',
         y = 'Sample Quantiles') +
    theme_classic() +
    theme(text = element_text(size = 15))
  qq_plots[[i]] <- qq_plot
}

grid.arrange(grobs = qq_plots, ncol = 4, widths = rep(2, 4))

In [ ]:
# Histograma del valor medio de precio de la vivienda
ggplot(data_boston, aes(x = median_value)) +
  geom_histogram(binwidth = 2, fill = "skyblue", color = "black") +
  labs(title = "Distribución del Valor Medio de Precio de la Vivienda",
       x = "Valor Medio de Precio (en 1000 USD)",
       y = "Frecuencia")

# Scatterplot de crimen vs. valor medio de precio
ggplot(data_boston, aes(x = crime_index, y = median_value)) +
  geom_point(color = "darkred") +
  labs(title = "Relación entre Índice de Criminalidad y Valor Medio de Precio",
       x = "Índice de Criminalidad",
       y = "Valor Medio de Precio (en 1000 USD)")


## 2. Análisis de correlación y de componentes principales (3p):
Se valorará principalmente la interpretación de dichos análisis así como la “limpieza” de la presentación
  

Análisis de correlación

In [ ]:
# Seleccionar las variables numéricas para el análisis de correlación
variables_numericas_all <- c("crime_index", "urban_ratio", "industry_ratio", "c_river_variable",
                         "nox_concentration", "rooms_per_dwelling", "dwelling_age",
                         "dwelling_dist", "highway_index", "tax_ratio", "teacher_ratio",
                         "racial_index", "lower_status", "median_value")

variables_numericas <- c("crime_index", "urban_ratio", "industry_ratio",
                         "nox_concentration", "rooms_per_dwelling", "dwelling_age",
                         "dwelling_dist", "highway_index", "tax_ratio", "teacher_ratio",
                         "racial_index", "lower_status", "median_value")
# Calcular la matriz de correlación
cor_matrix_boston <- cor(data_boston[, variables_numericas])

# Visualizar la matriz de correlación con corrplot
corrplot::corrplot(cor_matrix_boston, method = "circle")


In [ ]:
cov_matrix <- cov(data_boston_num)
cor_matrix <- cor(data_boston_num)

cat("Matriz de Covarianzas:\n")
#print(round(cov_matrix, 3))

cat("\nMatriz de Correlaciones:\n")
#print(round(cor_matrix, 3))

options(repr.plot.width = 15, repr.plot.height = 12)

ggpairs(
  data_boston_num,
  title = "Matrix with Correlation Coefficients",
  upper = list(continuous = wrap("cor", size = 9, color ='#ad3232'))
) +
theme_bw() +
theme(
  text = element_text(size = 12),
  axis.text = element_text(size = 10),
  axis.title = element_text(size = 15)
)


In [ ]:
# Visualización con ggpair#data_boston$c_river_variable,s
data_boston_key <- data.frame(
  data_boston$crime_index,
  data_boston$urban_ratio,
  data_boston$industry_ratio,

  data_boston$nox_concentration,
  data_boston$rooms_per_dwelling,
  data_boston$dwelling_age,
  data_boston$dwelling_dist,
  data_boston$highway_index,
  data_boston$tax_ratio,
  data_boston$teacher_ratio,
  data_boston$racial_index,
  data_boston$lower_status,
  data_boston$median_value
)
ggpairs(data_boston_key)

In [ ]:
# Calcular la matriz de correlación
cor_matrix_boston <- cor(data_boston_key)

# Obtener las 10 principales correlaciones
top_correlations <- as.data.frame(as.table(cor_matrix_boston))

top_correlations <- top_correlations[order(-abs(top_correlations$Freq)), ]

# Seleccionar las 10 principales correlaciones
top_correlations <- top_correlations[14:23, ]
print(top_correlations)
# Filtrar las variables correspondientes
variables_top_correlations <- unique(c(top_correlations$Var1, top_correlations$Var2))
data_boston_top_corr <- data_boston[, variables_top_correlations]

# Visualización con ggpairs
ggpairs(data_boston_top_corr)


In [ ]:
$ crime_index       : num  0.00632 0.02731 0.02729 0.03237 0.06905 ...
 $ urban_ratio       : num  18 0 0 0 0 0 12.5 12.5 12.5 12.5 ...
 $ industry_ratio    : num  2.31 7.07 7.07 2.18 2.18 2.18 7.87 7.87 7.87 7.87 ...
 $ c_river_variable  : int  0 0 0 0 0 0 0 0 0 0 ...
 $ nox_concentration : num  0.538 0.469 0.469 0.458 0.458 0.458 0.524 0.524 0.524 0.524 ...
 $ rooms_per_dwelling: num  6.58 6.42 7.18 7 7.15 ...
 $ dwelling_age      : num  65.2 78.9 61.1 45.8 54.2 58.7 66.6 96.1 100 85.9 ...
 $ dwelling_dist     : num  4.09 49.67 49.67 60.62 60.62 ...
 $ highway_index     : int  1 2 2 3 3 3 5 5 5 5 ...
 $ tax_ratio         : int  296 242 242 222 222 222 311 311 311 311 ...
 $ teacher_ratio     : num  15.3 17.8 17.8 18.7 18.7 18.7 15.2 15.2 15.2 15.2 ...
 $ racial_index      : num  397 397 393 395 397 ...
 $ lower_status      : num  4.98 9.14 4.03 2.94 5.33 ...
 $ median_value      : num  24 21.6 34.7 33.4 36.2 28.7 22.9 27.1 16.5 18.9 ...

In [ ]:
data_macro_key = data.frame(data_macro$Morosidad_prct
                            ,data_macro$PIB
                            ,data_macro$TasaParo_prct#<<
                            ,data_macro$IPC#<<
                            ,data_macro$Precio_avg_m2viviendas#<<
                            ,data_macro$Precio_vivienda_x_Rentabrutahogar#<<
                            ,data_macro$Tipo_cambio_EUR_USD
                            ,data_macro$Renta_dispo_hogares_mm)

ggpairs(data_macro_key)

cor(data_macro_key)
### Análisis de correlación
corrplot::corrplot(cor(data_macro_key)
                   , type = "upper" #para que solo muestre de la diagonal hacia arriba (la otra mitad es igual)
                   , number.cex = .6
                   , tl.col = "black"
                   , tl.srt = 90)

#PIB
a = ggplot(data_macro) +
  aes(x = PIB,y = Morosidad_prct,color = Morosidad_prct) +
   geom_point(size = 5,show.legend = FALSE) +
   scale_color_gradient(low = "#d94c9f", high = "#8c0052") +
   labs(title = "Boxplot Morosidad_prct x PIB " )
#TasaParo_prct
b = ggplot(data_macro) +
  aes(x = TasaParo_prct,y = Morosidad_prct,color = Morosidad_prct) +
   geom_point(size = 5,show.legend = FALSE) +
   scale_color_gradient(low = "#d94c9f", high = "#8c0052") +
   labs(title = "Boxplot Morosidad_prct x TasaParo_prct " )
#IPC
c = ggplot(data_macro) +
  aes(x = IPC ,y = Morosidad_prct,color = Morosidad_prct) +
   geom_point(size = 5 ,show.legend = FALSE) +
   scale_color_gradient(low = "#d94c9f", high = "#8c0052") +
   labs(title = "Boxplot Morosidad_prct x IPC " )
#Precio_avg_m2viviendas
d = ggplot(data_macro) +
  aes(x = Precio_avg_m2viviendas,y = Morosidad_prct,color = Morosidad_prct) +
   geom_point(size = 5,show.legend = FALSE) +
   scale_color_gradient(low = "#d94c9f", high = "#8c0052") +
   labs(title = "Boxplot Morosidad_prct x Precio_avg_m2viviendas " )
#Tipo_cambio_EUR_USD
e = ggplot(data_macro) +
  aes(x = Tipo_cambio_EUR_USD ,y = Morosidad_prct ,color = Morosidad_prct) +
   geom_point(size = 5,show.legend = FALSE) +
   scale_color_gradient(low = "#d94c9f", high = "#8c0052") +
   labs(title = "Boxplot Morosidad_prct x Tipo_cambio_EUR_USD " )
#Renta_dispo_hogares_mm
f = ggplot(data_macro) +
  aes(x = Renta_dispo_hogares_mm,y = Morosidad_prct,color = Morosidad_prct) +
   geom_point(size = 5,show.legend = FALSE) +
   scale_color_gradient(low = "#d94c9f", high = "#8c0052") +
   labs(title = "Boxplot Morosidad_prct x Renta_dispo_hogares_mm " )
grid.arrange(a,b,c,d,e,f)

In [ ]:
### Análisis de correlación
corrplot::corrplot(cor(data_macro_key)
                   , type = "upper" #para que solo muestre de la diagonal hacia arriba (la otra mitad es igual)
                   , number.cex = .6
                   , tl.col = "black"
                   , tl.srt = 90)# Calcular matriz de correlación
cor_matrix <- cor(data_boston)

# Mostrar matriz de correlación
print(cor_matrix)

# Gráfico de mapa de calor de la matriz de correlación
library(reshape2)
library(ggplot2)

ggplot(data = melt(cor_matrix), aes(x = Var1, y = Var2, fill = value)) +
  geom_tile(color = "white") +
  scale_fill_gradient2(low = "blue", high = "red", mid = "white", midpoint = 0) +
  theme_minimal() +
  labs(title = "Matriz de Correlación entre Variables")


In [ ]:
# Cargar biblioteca para PCA
library(FactoMineR)

# Seleccionar solo las variables numéricas para el PCA
datos_pca <- data_boston[, -c(4)]  # Excluir la variable c_river_variable que es categórica

# Estandarizar las variables para tener media 0 y desviación estándar 1
datos_pca_standardized <- scale(datos_pca)

# Realizar el análisis de componentes principales (PCA)
resultados_pca <- PCA(datos_pca_standardized, graph = FALSE)

# Resumen de los resultados del PCA
summary(resultados_pca)

# Gráfico de varianza explicada por cada componente principal
fviz_eig(resultados_pca, addlabels = TRUE, title = "Varianza Explicada por Componente Principal")

# Gráfico de individuos en el espacio de las dos primeras componentes principales
fviz_pca_ind(resultados_pca, habillage = datos_boston$c_river_variable, title = "PCA: Individuos")

# Gráfico de variables en el espacio de las dos primeras componentes principales
fviz_pca_var(resultados_pca, col.var = "contrib", gradient.cols = c("blue", "red"), title = "PCA: Variables")


## 3. Análisis de ANOVA (2p):
Se valorará principalmente la interpretación del análisis, así como la limpieza de los datos


## 4. Desarrollo y justificación de las variables (1p):
 En base a las conclusiones sacadas en los puntos 1, 2, y 3 ¿Qué variables debemos utilizar en nuestro modelo de regresión lineal?


## 5. Estudio de regresión lineal del median_value (2p):
¿Qué resultados obtenemos en el modelo?,  ¿Predice correctamente?, ¿Utilizarías este modelo en la vida real?